In [1]:
import re
import pymongo
import sys

# 以`col_wb_timePeriod_0`这1个小时的数据为例

## 1 选中目标集合

In [9]:
# 连接数据库
db = pymongo.MongoClient("mongodb://localhost:27017/")
db_Wb_SH0331 = db["db_Wb_SH0331"]

targetDB = 'db_Wb_SH0331'
targetCol = 'col_wb_timePeriod_3' # ----------------------------------------------改这个

# 选中目标数据库
exec(targetDB + " = db['" + targetDB + "']")
print(targetDB + ": 已连接！")

# 选中目标集合
exec(targetCol + " = " + targetDB + "['" + targetCol + "']")
print("当前集合：" + targetCol)


exec("nums_recs = " + targetCol + ".count_documents({})")

# 查看目标集合的记录数
loc = locals()
total_count = loc['nums_recs']
print("===================================")
print(f"总记录数：{total_count}")

db_Wb_SH0331: 已连接！
当前集合：col_wb_timePeriod_3
总记录数：10741


## 2 新身份生成函数

In [6]:
# 计算一个字符串的新"身份"
def extract5Chars(content):
    
    length = len(content)
    
    if(length == 0):
        return ""
    
    selectLoc = [0, 0.25, 0.5, 0.75]
    idx = [int(elem * length) for elem in selectLoc]
    
    newKey = ""
    # 分别获取这个字符的第0/4号、第1/4号，第2/4号，第3/4号位置的字符
    for i in idx:
        newKey = newKey + content[i]
    # 追加上最后一个字符，即，第4/4号位置字符
    newKey = newKey + content[-1]
    
    return newKey

## 3 在目标集合内，为每个记录生成一个新身份
* 在获取每条记录的时候，直接判断字段名有没有，没有的话，直接操作新字段（就等于新建了）

In [10]:
progressBar = 0
for eachRec in col_wb_timePeriod_3.find():
    progressBar = progressBar + 1
    if progressBar % 50000 == 0:
        print("已处理" + str(progressBar) + "条记录！")
    id = eachRec['_id']
    orgContent = eachRec['orgContent']
    newKey = extract5Chars(orgContent)
    newField = "newKey"

    if newField not in eachRec.keys():
        col_wb_timePeriod_3.update_one({'_id': id}, {'$set': {newField: newKey}})


print("处理完毕，共处理" + str(progressBar) + "条记录！")

处理完毕，共处理10741条记录！


## 4 在目标集合内，统计频数

In [11]:
orgContentDic = {}

progressBar = 0
for eachRec in col_wb_timePeriod_3.find():
    progressBar = progressBar + 1
    if progressBar % 50000 == 0:
        print("已处理" + str(progressBar) + "条记录！")
        
    newKey = eachRec['newKey']
    orgContentDic[newKey] = orgContentDic.get(newKey, 0) + 1  # 统计每个newKey的频数
        
items = list(orgContentDic.items())  # 转换为list


total_num = len(items)  # 不一样的对象的个数
print("items.len = " + str(total_num)) 

items.sort(key=lambda x: x[1], reverse=True)  # 按照频数排序

docDict = dict()  
for i in range(20):
    doc, count = items[i]
    docDict[doc] = count
    print("{0:<20}{1:>10}".format(doc, count))

items.len = 1142
2救，的x                     4539
                          1013
#，、真?                      344
#挤舔绝?                      291
最的价??                      231
#酸，院F                      205
这-伶了?                      177
#周人时坊                      161
刚的量事。                      106
坐，评手?                      100
实日等上?                       95
2都饿]?                       66
【金一还#                       57
我流可待歌                       54
#希下上?                       53
【0的东#                       51
只越家不#                       46
我原核涩?                       45
图，，，?                       39
这看:t?                       37


#### 上面的输出里，有个是“null”，计数是1013，应该是“原创”微博（没有转发别人的）
* 我们获取当前集合的原创微博的数量看看，确认一下，果然是对应的！

In [12]:
queryCondition = {"OrgOrFwd": "原创"}
col_wb_timePeriod_3.count_documents(queryCondition)

1013

## 5 获取频率字典的第一个元素（排名最高的）

In [16]:
print(docDict)

key1 = next(iter(docDict))

print("first key : ", key1)
print("first value : ", docDict.get(key1))

{'2救，的x': 4539, '': 1013, '#，、真?': 344, '#挤舔绝?': 291, '最的价??': 231, '#酸，院F': 205, '这-伶了?': 177, '#周人时坊': 161, '刚的量事。': 106, '坐，评手?': 100, '实日等上?': 95, '2都饿]?': 66, '【金一还#': 57, '我流可待歌': 54, '#希下上?': 53, '【0的东#': 51, '只越家不#': 46, '我原核涩?': 45, '图，，，?': 39, '这看:t?': 37}
first key :  2救，的x
first value :  4539


#### 查看doc详情

In [22]:
queryCondition2 = {"newKey": key1}
temp = col_wb_timePeriod_3.find(queryCondition2)
# for each in temp:
#     print(each["orgContent"])
print(type(temp))
print("-------------------------")
print(temp[0]["newKey"])
print("-------------------------")
print(temp[0]["orgContent"])

<class 'pymongo.cursor.Cursor'>
2救，的x
2022年3月30日星期三，上海市浦东新区航昌路376弄瑞祥苑上午8点左右，804室有人哮喘发作，用了气雾剂，未好转，出现危险，打120；8点10分到8点30分，邻居帮忙救治，打120，110；8点30分 负责做核酸检测的大白到达，做心肺复苏。8:30-9:40 之间，具体时间未知：邻居下楼找小区大门口的110警车寻求帮助，但是他们说：“工作地点和工作任务就在这里，不能走，不能动”。有一辆120救护车到达小区，但是来救助202室病人的，不是救助804的。救护人员得知804需要急救，拒不出借除颤仪，也未提供任何帮助，接到202号病人后离去。9点40分 第二辆120救护车到场，抢救无效，804病人已过世。上午11点左右，殡仪车到场。上午12点左右，殡仪车带着过世病人离开。这位哮喘病人，就住在我楼下，经常见面，每次都是微笑着打招呼。他很和善，经常拿猫粮和小鱼喂养小区的流浪猫。开完视频会议，惊闻他因这种情况离世，伤心，同时也很愤怒。参与救助的邻居：“当放下无能为力的双手的时候，心都碎了。眼睁睁看着生命的流逝是一种这怎样的心情。”当你心急如焚地找警察帮助，警车就停在小区门口，却对需要急救的人熟视无睹，反而恪尽职守地“守在原地”，这是怎么样的心情？当你终于看到120急救车停在楼下，赶紧跑过去寻求帮助，却得知这是救助别人的车，不管你家人的死活，连除颤仪也不外借，这是怎样的心情？2022年新冠疫情仍在肆虐，但是为了抗疫，我们究竟要付出多大代价？2022年3月11日下午，吉林省长春市农安县某村的4岁女童急性喉炎发作，因疫情关系在医院不同科室奔走而未得到及时救治，离世。2022年3月23日上海东方医院的护士周盛妮哮喘发作，用药后无法缓解。其所就职的医院及附近的几所医院急诊部因疫情防控需要在关闭，未得到及时救治，离世。2022年3月30日，上海浦东瑞祥苑病人因哮喘发作，警察和某救护车各司其职，未及时进行救治，病人离世。因疫情，浦东封控，足不出户，药店关闭。隔壁癌症病人无法去浦西就诊的医院继续做化疗，也有病人因疫情无法去医院做血液透析，说好的绿色通道呢？12345热线已经被打爆了，真的需要12345挨家挨户解决吗？。想起前两天上海某位爸爸的做法：4岁的孩子摔了一跤，磕到后脑勺，不久后开始呕吐。联系120无果后，决定

## 6 下面获取当前集合的所有记录的转发路径

In [23]:
progressBar = 0
for eachRec in col_wb_timePeriod_3.find():
    progressBar = progressBar + 1
    if progressBar % 50000 == 0:
        print("已处理" + str(progressBar) + "条记录！")
    id = eachRec['_id']
    title8content = eachRec['title8content']

    patternRepost = re.compile(r"\/@(.+?)\:")
    
    
    repostNet2 = patternRepost.findall(title8content)  # 提取title8content中的符合"/@"与符号":"之间的用户，构成转发网络
    
    # 相较于上一行代码，这里去除了提取得到的用户的前后空格，即["aaa ", " bbb "] → ["aaa", "bbb"]
    repostNet3 = [each.strip() for each in repostNet2]

    newField = "repostNet3"

    if newField not in eachRec.keys():
        col_wb_timePeriod_3.update_one({'_id': id}, {'$set': {newField: repostNet3}})


print("处理完毕，共处理" + str(progressBar) + "条记录！")

处理完毕，共处理10741条记录！


### 6.1 下面获取一个固定目标的所有记录，提取所有记录的转发路径（大于等于2的才要）

#### 功能函数：这4个函数是为Gephi定制的格式

In [54]:
def generateCSV_string_node(dic):
    s = ''
    for key, value in dic.items():
        s = s + key + ',' + str(value) + '\n'
    return s

# 边太多了，给一个过滤参数（默认：0）
def generateCSV_string_edge(dic, freqNum = 0):
    total = len(dic.keys())
    print("===================================")
    print(f"总数：{total}")
    cnt = 0
    s = ''
    for key, value in dic.items():
        cnt = cnt + 1
        if cnt % 50000 == 0:
            print(f"已处理 / 总数: {cnt} / {total}")

        if value > freqNum:
            s = s + key + ',' + str(value) + '\n'
    return s

def str2csv_node(filePath, s):
    '''
    将字符串写入到本地csv文件中
    :param filePath: csv文件路径
    :param s: 待写入字符串(逗号分隔格式)
    '''
    with open(filePath, 'w', encoding='utf-8') as f:
        f.write("Label,Weight\r")
        f.write(s)
    print('写入文件成功,请在'+filePath+'中查看')
    
def str2csv_edge(filePath, s):
    '''
    将字符串写入到本地csv文件中
    :param filePath: csv文件路径
    :param s: 待写入字符串(逗号分隔格式)
    '''
    with open(filePath, 'w', encoding='utf-8') as f:
        f.write("Source,Target,Weight\r")
        f.write(s)
    print('写入文件成功,请在'+filePath+'中查看')

In [25]:
# 首先，获取固定newKey的所有记录
# 然后遍历所有记录，提取repostNet3字段
# 判断，字段长度，大于等于2才执行

docID = "2救，的x"
queryCondition = {"newKey": docID}
curWordRecs = col_wb_timePeriod_3.find(queryCondition)

targetField = "repostNet3"

node_dict = {}  # 节点字典,包含节点名+节点权值(频数)
edge_dict = {}  # 边字典,包含起点+目标点+边权值(频数)

for cW in curWordRecs:
    row_authors_list = cW.get(targetField)
    
    len_list = len(row_authors_list)
    if len_list >= 2:
        rev_list = row_authors_list[::-1]
        print(rev_list)
        print("==========")
        for each in rev_list:
            node_dict[each] = node_dict.get(each, 0) + 1
        
        for i in range(len_list - 1):
            A,B = rev_list[i], rev_list[i+1]
            key = A+','+B  # 格式化为逗号分隔A,B形式,作为字典的键
            
            edge_dict[key] = edge_dict.get(key, 0) + 1

['秋秋也是阿秀', '在也门钓鲑鱼', '穷小疯']
['十肆桥', '票圈君', '养鸡大王欧阳铁柱', '快递员吴彦祖', 'M大王叫我来巡山', '江南大野花', '胭脂牌锦鲤']
['秋秋也是阿秀', '-岚舞-', '青锋裁月', '专业戳轮胎熊律师']
['秋秋也是阿秀', '壁纸酱bot']
['秋秋也是阿秀', '今天我该吃什么', '壑舟无须臾']
['秋秋也是阿秀', 'G僧东']
['佛心蛊', '他们叫我冰箱', '客官不要急', 'DDG的老船坞', '近卫T-34', '踏踏实实的熊主任']
['秋秋也是阿秀', 'Sistersinlaw']
['马瑞连焖肉', '逃亡者的恰恰', '小钟燚', '黑皮辣t本t']
['秋秋也是阿秀', '田螺姑娘_hhhaze']
['秋秋也是阿秀', '常世-透明']
['Regene_53408', '演播室只有一个屎蛋', 'MONK_WALZ']
['秋秋也是阿秀', 'H-urchin']
['庄时利和', '为格命思奔']
['来去之间', 'flypig', '差评君', '卡利斯厨']
['十肆桥', '票圈君', '养鸡大王欧阳铁柱', '快递员吴彦祖', 'M大王叫我来巡山', '江南大野花', '设计系奶子']
['Regene_53408', '演播室只有一个屎蛋']
['中杯热澳白加一份浓缩谢谢', '拷扁橄榄', '大力豆是大横按肌无力患者', 'Biesan哈哈哈']
['陈迪Winston', '王磬']
['秋秋也是阿秀', '·口羊羊·', '-夢境劇院-']
['我不是盆栽啊', '马瑞连焖肉', '逃亡者的恰恰', '这是谁在胡说八道']
['十肆桥', '票圈君', '养鸡大王欧阳铁柱', '快递员吴彦祖']
['HHH韩夏', 'Yue226']
['苦海游鱼', 'sjszcmdr']
['秋秋也是阿秀', 'G僧东']
['KATRIN', '大宝', '藤原定子']
['十肆桥', '票圈君', '养鸡大王欧阳铁柱', '快递员吴彦祖', 'M大王叫我来巡山']
['赫Hae同行人_', '希净葉子', 'Gashinisme', '甜甜小pupu']
['秋秋也是阿秀', '徐小狙', '搞垮当代文字博主']
['秋秋也是阿秀', 'Mashooo

['秋秋也是阿秀', '-夢境劇院-']
['秋秋也是阿秀', 'internet-garcon-pdf']
['十肆桥', '票圈君', '养鸡大王欧阳铁柱', '取名字很难但是还是想换', '笨蛋君不笨蛋']
['十肆桥', '羊爱零', '路尼尼finale']
['怡然大大宝', '-冯冯冯冯']
['白日好梦丨', '影子在身后月光在心里', '爱橘人士biu']
['秋秋也是阿秀', '罗匡威']
['秋秋也是阿秀', 'magasa', '何小沁', '胡晋晗', '赵大宝', '214-', '线头女', 'boihugo', '祝狍子']
['sunwear', '黑洞胃竹竿身', '斯卡亨侍Lv90']
['秋秋也是阿秀', '在也门钓鲑鱼', '西窗随记']
['ChineseRock摇滚地狱', '周楊HOPICO']
['来去之间', 'flypig', '贾伯斯龙', '老七']
['秋秋也是阿秀', 'thndrP-mod3']
['秋秋也是阿秀', 'G僧东']
['秋秋也是阿秀', '自然男孩儿']
['不绿不蓝', '柚子Yuzu皮', '蛾啽寺', '小蓝三号机']
['秋秋也是阿秀', '赛车星冰乐']
['德川狸猫姬', 'Fydia_Ume', 'daaaaaaahlia', 'miyuki_hyde', 'satoru一心求二巡']
['秋秋也是阿秀', 'magasa', '何小沁', '胡晋晗', '赵大宝', '王梓777', 'GusRubato']
['秋秋也是阿秀', '计六一六', '张尕怂']
['我不是盆栽啊', '马瑞连焖肉', '逃亡者的恰恰', '小钟燚', '糊糊不是FUFU', '小菜和小李要认真生活', '无情老彩狗']
['海牛蕾姆', '温带菠萝树_']
['秋秋也是阿秀', 'special特特']
['秋秋也是阿秀', '爱咋咋地没救了']
['秋秋也是阿秀', '红鹤鸣冬']
['秋秋也是阿秀', '赏味不足', 'Rebecca是小圆子吖', '老刀99']
['庄时利和', '为格命思奔']
['秋秋也是阿秀', '樂樂攣']
['子健RockZO', '解说老陈']
['秋秋也是阿秀', '小阳宝莉']
['秋秋也是阿秀', 'yokoego']
['秋秋也是阿秀', '豆你

In [89]:
# 查看 node_dict 的第一个
# 查看 edge_dict 的第一个

n1 = next(iter(node_dict))

print(str(n1) + "|" + str(node_dict.get(n1)))

e1 = next(iter(edge_dict))

print(str(e1) + "|" + str(edge_dict.get(e1)))

秋秋也是阿秀|365
秋秋也是阿秀,在也门钓鲑鱼|16


### 6.2 Gephi处理的目标文件已生成

In [26]:
node_csv_string = generateCSV_string_node(node_dict)
edge_csv_string = generateCSV_string_edge(edge_dict, 0)

filePath1 = f'E:/workspace/pyWorkspace/weiboDataPrograms/repostMatrix/col3_gephi_node_repostNet3.csv'
filePath2 = f'E:/workspace/pyWorkspace/weiboDataPrograms/repostMatrix/col3_gephi_edge_repostNet3.csv'

# 将字符串写入到本地csv文件中
str2csv_node(filePath1,node_csv_string)
print("[node] write done!")
str2csv_edge(filePath2,edge_csv_string)
print("[edge] write done!")

总数：546
写入文件成功,请在E:/workspace/pyWorkspace/weiboDataPrograms/repostMatrix/col3_gephi_node_repostNet3.csv中查看
[node] write done!
写入文件成功,请在E:/workspace/pyWorkspace/weiboDataPrograms/repostMatrix/col3_gephi_edge_repostNet3.csv中查看
[edge] write done!


### 6.3 分层图的json数据生成 & 绘制（大圆套小圆）
* 从node_dict里提取`name`,`size`
* 从edge_dict里提取`imports`

In [62]:
def save_variable2(v, filename):
    try: 
        # f = open('test001.txt','w',encoding='utf-8') #open 打开要写入的文件，'w'就是写入的意思，如果有中文encoding进行编码
        with open(filename,'w',encoding='utf-8') as f:
            f.write(v)  # write是写入的意思，将（）中的内容写入到f文件里
            
    # 无论try中的程序是否存在报错，则都执行下面的关闭
    finally: 
        if f:  # 判断下f是否打开了，如果没打开则不需要关闭，打开了则关闭
            f.close()
    return filename

#### 6.3.1 生成json数据并保存

In [75]:
import json


dheb_json = []

for name, count in node_dict.items():
    
    temp = {}
    
    temp["name"] = name
    temp["size"] = count
    temp["imports"] = list()
    
    for key in edge_dict.keys():
        edge = key.split(',')
        
        if name == edge[0]:
            temp["imports"].append(edge[1])

    dheb_json.append(json.dumps(temp,ensure_ascii=False))
    
# 随便查看一个数据对象
print(dheb_json[0])

res_json = ""
for obj in dheb_json:
    res_json = res_json + obj + ','
    
res_json = "[" + res_json[:-1] + "]"
# print(res_json)

filePathCircle = f'E:/workspace/pyWorkspace/weiboDataPrograms/nets/dheb_json.json'
save_variable2(res_json, filePathCircle)


{"name": "秋秋也是阿秀", "size": 365, "imports": ["在也门钓鲑鱼", "-岚舞-", "壁纸酱bot", "今天我该吃什么", "G僧东", "Sistersinlaw", "田螺姑娘_hhhaze", "常世-透明", "H-urchin", "·口羊羊·", "徐小狙", "Mashooo", "孟常", "Kureha003", "Shu刘树伟", "施乐遥", "_Masa_", "LeslieZhangJiacheng", "magasa", "纯洁白果酱", "噢小老王", "罐头辰", "sven_shi", "假装在纽约", "陀螺的ToroScope", "ZGnneF", "河汉", "RefreshLUp", "自然男孩儿", "欧洲文艺评论", "爱咋咋地没救了", "奏耐天津", "作死狂喵", "颜文字君", "打狗诗人", "FIMA-Repo", "seabisuit", "姐姐甩笔都不甩你", "鋼筆様子", "四包包包包", "芋圆牛奶厨", "貝拉一九九九", "北国佳人李春姬", "疑夕", "IceHex_", "无边桃炎", "李晓鹏1982", "豆你玩", "毛十八", "殳俏", "刀卜卜", "longago-faraway", "修老虎", "董晨宇RUC", "杨砚青", "来去之间", "浪漫冬季里的敏", "Hey是SuSu啊", "狂很子老", "aoistudio青设", "霸道大锤爱上我", "初号渣", "废几把话", "朱海舟", "渥丹", "红鹤鸣冬", "胜虎次郎", "郭玉洁", "狠狠红", "琉璃厂人", "人律法", "IsurT_", "internet-garcon-pdf", "暖小团", "校花驴", "简木生--撰稿者", "哎呀我摔倒辣", "反吃瓜联盟", "Yuukii_c", "山河草莓", "internet-gadf", "比格犬受害者联盟", "素质美女鹅", "哈三五四青年V", "保守主义评论", "野生11", "Crisisa__", "十二月断片", "成都市精神内科副主任", "桃阿蛮", "ProfessorX_", "在下翩竹", "Cloud61", "hoooooOHaaaaa", "D2O___", 

'E:/workspace/pyWorkspace/weiboDataPrograms/nets/dheb_json.json'

#### 6.3.2 分层图代码
`E:\坚果云t\0 学习 - web开发\VSCode workspace\prj_mapCon\placesUsameWeibo\hierarchical.html`

### 6.4 Network的json数据生成 & 绘制（网络图）

#### 6.4.1 nodes 搞定
`{"id": 1, "name": "秋秋也是阿秀", "weight": 365}`

In [90]:
import json

nodeJson = []
id = 0

name2id = {}
for name,weight in node_dict.items():
    temp = {}
    id = id + 1
    temp["id"] = id
    temp["name"] = name
    temp["weight"] = weight
    name2id[name] = id
    
    nodeJson.append(json.dumps(temp, ensure_ascii=False))

In [91]:
resNode_json = ""
for eee in nodeJson:
    resNode_json = resNode_json + eee + ','
    
resNode_json = "[" + resNode_json[:-1] + "]"

#### 6.4.2 edges 搞定
`{"source": 1, "target": 2, "weight": 16}`

In [92]:
edgeJson = []

for key,weight in edge_dict.items():
    temp = {}
    
    sota = key.split(',')
    temp["source"] = name2id.get(sota[0])
    temp["target"] = name2id.get(sota[1])
    temp["weight"] = weight
    
    edgeJson.append(json.dumps(temp, ensure_ascii=False))

In [93]:
resEdge_json = ""
for eee in edgeJson:
    resEdge_json = resEdge_json + eee + ','
    
resEdge_json = "[" + resEdge_json[:-1] + "]"

#### 6.4.3 合并
```json
{
  "nodes": [
    {
      "id": 1,
      "name": "秋秋也是阿秀",
      "weight": 365
    },
    ...
    {
      "id": 611,
      "name": "未必存在已不存在",
      "weight": 1
    }
  ],
  "edges": [
    {
      "source": 1,
      "target": 2,
      "weight": 16
    },
    ...
    {
      "source": 2,
      "target": 3,
      "weight": 1
    }
  ]
}
```

In [95]:
finalRes = '{"nodes": ' + resNode_json + ',"edges": ' + resEdge_json + '}'

filePathNet = f'E:/workspace/pyWorkspace/weiboDataPrograms/nets/data_networks.json'
save_variable2(finalRes, filePathNet)

'E:/workspace/pyWorkspace/weiboDataPrograms/nets/data_networks.json'

## 7 下面获取当前集合的指定微博的信源城市

### 7.1 获取拥有相同的“原始微博”的所有记录的“信源城市”
* 获取城市名 & 频数

In [35]:
docID = "2救，的x"
queryCondition = {"newKey": docID}
curWordRecs = col_wb_timePeriod_3.find(queryCondition)

regionDic = {}
for each in curWordRecs:
    key = each.get('SourceRegion')
    regionDic[key] = regionDic.get(key, 0) + 1

items = list(regionDic.items())  # 转换为list


total_num = len(items)  # 不一样的对象的个数
print("items.len = " + str(total_num)) 

items.sort(key=lambda x: x[1], reverse=True)  # 按照频数排序

cityDict = dict()  
for i in range(total_num):
    city, count = items[i]
    cityDict[city] = count
    print("{0:<20}{1:>10}".format(city, count))
    
    
s = ''

for city in items:
    s = s + city[0] + '\n'

filename = f'E:/workspace/pyWorkspace/weiboDataPrograms/places/col3_allCityName.txt'

save_variable2(s, filename)

items.len = 34
北京                        1478
上海                         961
广东                         450
江苏                         265
浙江                         180
福建                         132
山东                          99
四川                          91
广西                          81
安徽                          70
湖南                          69
辽宁                          68
湖北                          67
重庆                          64
河南                          63
河北                          63
天津                          48
陕西                          43
江西                          37
吉林                          30
云南                          29
黑龙江                         20
香港                          18
山西                          18
新疆                          17
贵州                          16
海南                          15
甘肃                          14
内蒙古                         12
澳门                           6
青海                           6
台湾                      

'E:/workspace/pyWorkspace/weiboDataPrograms/places/col3_allCityName.txt'

### 7.2 下面把城市的出现频数，换算一下，变小点

In [33]:
import pandas as pd

freqList = []  # 每个城市的频数(已排序的)
total = 0  # 城市出现的总数
for i in range(0, total_num):
    total = total + items[i][1]  # 累加总次数
    
    freqList.append(items[i][1])
    
print(freqList)
print([n/total for n in freqList])

[1478, 961, 450, 265, 180, 132, 99, 91, 81, 70, 69, 68, 67, 64, 63, 63, 48, 43, 37, 30, 29, 20, 18, 18, 17, 16, 15, 14, 12, 6, 6, 4, 3, 2]
[0.32562238378497466, 0.21172064331350518, 0.0991407799074686, 0.05838290372328707, 0.03965631196298744, 0.029081295439524125, 0.021810971579643092, 0.02004846882573254, 0.01784534038334435, 0.015421899096717338, 0.01520158625247852, 0.0149812734082397, 0.01476096056400088, 0.014100022031284423, 0.013879709187045605, 0.013879709187045605, 0.010575016523463317, 0.009473452302269222, 0.008151575236836307, 0.006609385327164574, 0.006389072482925755, 0.004406256884776382, 0.0039656311962987445, 0.0039656311962987445, 0.003745318352059925, 0.003525005507821106, 0.003304692663582287, 0.0030843798193434676, 0.0026437541308658294, 0.0013218770654329147, 0.0013218770654329147, 0.0008812513769552765, 0.0006609385327164573, 0.0004406256884776382]


In [31]:
processFreqList = []

for each in freqList:
    temp = each / total
    if(temp > 0.009):
        processFreqList.append(round(temp * 100)) # 0.229423 → 22
    else:
        processFreqList.append(1) #  0.003486 → 1
    
cnt_obj = pd.Series(processFreqList)
print(cnt_obj)

print(total)

0     33
1     21
2     10
3      6
4      4
5      3
6      2
7      2
8      2
9      2
10     2
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
30     1
31     1
32     1
33     1
dtype: int64
4539


### 7.3 cityName → cityGPS
[在线转换](https://www.piliang.tech/geocoding-amap)

`E:/workspace/pyWorkspace/weiboDataPrograms/places/col3_allCityName_map-location.xlsx`

In [43]:
import pandas as pd
 
df = pd.read_excel(r'E:/workspace/pyWorkspace/weiboDataPrograms/places/col3_allCityName_map-location.xlsx')
data = df.values
# print(df.iloc[0])
print(df.head(1))

df.insert(0,'频数', cnt_obj) 
print(df.head(1))


   地址                 纬度,经度  坐标系 解析结果  错误信息
0  北京  116.407526,39.904030  NaN  北京市   NaN
   频数  地址                 纬度,经度  坐标系 解析结果  错误信息
0  33  北京  116.407526,39.904030  NaN  北京市   NaN


In [44]:
sll = ''
for eR in df.values:
    longlat = eR[2].split(',')
    print(ll)

['116.407526', '39.904030']
['121.473701', '31.230416']
['113.266530', '23.132191']
['118.763232', '32.061707']
['120.152791', '30.267446']
['119.295144', '26.100779']
['117.020359', '36.668530']
['104.075931', '30.651651']
['108.327546', '22.815478']
['117.284922', '31.861184']
['112.983810', '28.112444']
['123.429440', '41.835441']
['114.341861', '30.546498']
['106.551556', '29.563009']
['113.753602', '34.765515']
['114.468664', '38.037057']
['117.200983', '39.084158']
['108.954239', '34.265472']
['115.909228', '28.675696']
['125.325990', '43.896536']
['102.710002', '25.045806']
['126.661669', '45.742347']
['114.163825', '22.276284']
['112.562398', '37.873531']
['87.627704', '43.793026']
['106.707410', '26.598194']
['110.349228', '20.017377']
['103.826308', '36.059421']
['111.765617', '40.817498']
['113.543028', '22.186835']
['101.780199', '36.620901']
['121.508903', '25.044319']
['91.117212', '29.646922']
['106.258754', '38.471317']


`longitude` n.经度，经线<br/>
`latitude` n.纬度；纬度地区；选择的自由，回旋余地；（感光乳剂，相纸）曝光宽容度<br/>

* 地址 | 经度(`longitude`) | 纬度(`latitude`)    
* 上海  121.473701 | 31.230416

### 7.4 地图bubble文件已生成（经度 + 维度 + 频数）
* 符合d3.js的数据格式 → 
`E:/workspace/pyWorkspace/weiboDataPrograms/places/col3_allCityName_map-GPS.txt`

In [53]:
str_ll = ''
for eR in df.values:
    ll = eR[2].split(',')  # 获取GPS（注意顺序，网页导出的经纬度顺序有问题）

    line = "{long: " + ll[0] + ", lat:" + ll[1] + ", cnt:" + str(eR[0]) + "},\n"
    str_ll = str_ll + line

json_ll = "[" + str_ll[:-2] + "]"
print(json_ll)
filename = f'E:/workspace/pyWorkspace/weiboDataPrograms/places/col3_allCityName_map-GPS.txt'
save_variable2(json_ll, filename)

[{long: 116.407526, lat:39.904030, cnt:33},
{long: 121.473701, lat:31.230416, cnt:21},
{long: 113.266530, lat:23.132191, cnt:10},
{long: 118.763232, lat:32.061707, cnt:6},
{long: 120.152791, lat:30.267446, cnt:4},
{long: 119.295144, lat:26.100779, cnt:3},
{long: 117.020359, lat:36.668530, cnt:2},
{long: 104.075931, lat:30.651651, cnt:2},
{long: 108.327546, lat:22.815478, cnt:2},
{long: 117.284922, lat:31.861184, cnt:2},
{long: 112.983810, lat:28.112444, cnt:2},
{long: 123.429440, lat:41.835441, cnt:1},
{long: 114.341861, lat:30.546498, cnt:1},
{long: 106.551556, lat:29.563009, cnt:1},
{long: 113.753602, lat:34.765515, cnt:1},
{long: 114.468664, lat:38.037057, cnt:1},
{long: 117.200983, lat:39.084158, cnt:1},
{long: 108.954239, lat:34.265472, cnt:1},
{long: 115.909228, lat:28.675696, cnt:1},
{long: 125.325990, lat:43.896536, cnt:1},
{long: 102.710002, lat:25.045806, cnt:1},
{long: 126.661669, lat:45.742347, cnt:1},
{long: 114.163825, lat:22.276284, cnt:1},
{long: 112.562398, lat:37.8735

'E:/workspace/pyWorkspace/weiboDataPrograms/places/col3_allCityName_map-GPS.txt'

### 7.5 可视化代码（不需要导入外部数据）

```html
<!DOCTYPE html>
<meta charset="utf-8">

<!-- Load d3.js and the geo projection plugin -->
<script src="https://d3js.org/d3.v4.js"></script>
<script src="https://d3js.org/d3-geo-projection.v2.min.js"></script>

<!-- Create an element where the map will take place -->
<svg id="my_dataviz" width="800" height="700"></svg>

<script>

    // The svg
    var svg = d3.select("svg"),
        width = +svg.attr("width"),
        height = +svg.attr("height");
    
    // Map and projection
    var projection = d3.geoMercator()
        .center([112, 32])                // GPS of location to zoom on
        // scale越大，地图越大
        .scale(500)                       // This is like the zoom
        .translate([ width/2, height/2 ])
    
    // data from: E:/workspace/pyWorkspace/weiboDataPrograms/places/col3_allCityName_map-GPS.txt
    // Create data for circles:
    var markers = [{long: 116.407526, lat:39.904030, cnt:33},
{long: 121.473701, lat:31.230416, cnt:21},
{long: 113.266530, lat:23.132191, cnt:10},
{long: 118.763232, lat:32.061707, cnt:6},
{long: 120.152791, lat:30.267446, cnt:4},
{long: 119.295144, lat:26.100779, cnt:3},
{long: 117.020359, lat:36.668530, cnt:2},
{long: 104.075931, lat:30.651651, cnt:2},
{long: 108.327546, lat:22.815478, cnt:2},
{long: 117.284922, lat:31.861184, cnt:2},
{long: 112.983810, lat:28.112444, cnt:2},
{long: 123.429440, lat:41.835441, cnt:1},
{long: 114.341861, lat:30.546498, cnt:1},
{long: 106.551556, lat:29.563009, cnt:1},
{long: 113.753602, lat:34.765515, cnt:1},
{long: 114.468664, lat:38.037057, cnt:1},
{long: 117.200983, lat:39.084158, cnt:1},
{long: 108.954239, lat:34.265472, cnt:1},
{long: 115.909228, lat:28.675696, cnt:1},
{long: 125.325990, lat:43.896536, cnt:1},
{long: 102.710002, lat:25.045806, cnt:1},
{long: 126.661669, lat:45.742347, cnt:1},
{long: 114.163825, lat:22.276284, cnt:1},
{long: 112.562398, lat:37.873531, cnt:1},
{long: 87.627704, lat:43.793026, cnt:1},
{long: 106.707410, lat:26.598194, cnt:1},
{long: 110.349228, lat:20.017377, cnt:1},
{long: 103.826308, lat:36.059421, cnt:1},
{long: 111.765617, lat:40.817498, cnt:1},
{long: 113.543028, lat:22.186835, cnt:1},
{long: 101.780199, lat:36.620901, cnt:1},
{long: 121.508903, lat:25.044319, cnt:1},
{long: 91.117212, lat:29.646922, cnt:1},
{long: 106.258754, lat:38.471317, cnt:1}];

    // Load external data and boot
    d3.json("https://raw.githubusercontent.com/holtzy/D3-graph-gallery/master/DATA/world.geojson", function(data){
    
        // Filter data
        data.features = data.features.filter( function(d){return d.properties.name=="China"} )
    
        // Draw the map
        svg.append("g")
            .selectAll("path")
            .data(data.features)
            .enter()
            .append("path")
              .attr("fill", "#b8b8b8")
              .attr("d", d3.geoPath()
                  .projection(projection)
              )
            .style("stroke", "black")
            .style("opacity", .3)
    
        // Add circles:
        svg
          .selectAll("myCircles")
          .data(markers)
          .enter()
          .append("circle")
            .attr("cx", function(d){ 
                console.log(projection([d.long, d.lat])[0])
                return projection([d.long, d.lat])[0] 
            })
            .attr("cy", function(d){ return projection([d.long, d.lat])[1] })
            // .attr("r", 4)
            .attr("r", function(d){ return d.cnt })
            .style("fill", "69b3a2")
            .attr("stroke", "#69b3a2")
            .attr("stroke-width", 3)
            .attr("fill-opacity", .4)
    })
    
    </script>

```